In [0]:
from pyspark.sql.functions import col

In [0]:
damage_df = spark.table("02_silver.warcraftlogs.tables_damage_abilities_nested")  # columns: report_id, fight_id, player_id, total_damage, dps
fights_df = spark.table("02_silver.warcraftlogs.fights_boss_pulls")  # columns: report_id, fight_id, boss_id, boss_name, difficulty, kill, start_time
actors_df = spark.table("02_silver.warcraftlogs.actors_players")  # columns: report_id, player_id, player_name, class, spec, realm

In [0]:
display(damage_df)
display(fights_df)
display(actors_df)

In [0]:
print(f'damage_df columns: {damage_df.columns}')
print(f'fights_df columns: {fights_df.columns}')
print(f'actors_df columns: {actors_df.columns}')

In [0]:
df_damage_done = (
    df_flat
    .select("report_id", "report_start_date", col("fight").alias("pull_number"))
    .withColumn("player_name", F.col("name"))
    .withColumn("player_class", F.col("type"))
    .withColumn("event", F.explode_outer("events"))
    .filter(F.col("event.type") == "damage")
    .groupBy("report_id", "report_start_date", "pull_number", "player_name", "player_class")
    .agg(
        F.sum("event.amount").alias("total_damage")
    )
)

damage_df = 

In [0]:
# Join damage with fight and actor metadata
damage_with_fights = (
    damage_df.alias("dmg")
    .join(
        fights_df.alias("f"),
        on=[
            damage_df["report_id"] == fights_df["report_id"],
            damage_df["pull_number"] == fights_df["pull_number"],
            damage_df["report_date"] == fights_df["report_date"],
        ],
        how="inner"
    )
    .join(
        actors_df.alias("a"),
        on=[
            damage_df["report_id"] == actors_df["report_id"],
            damage_df["player_name"] == actors_df["player_name"],
            damage_df["report_date"] == actors_df["report_date"]
        ],
        how="inner"
    )
    .select(
        "dmg.report_id", "dmg.report_date", "dmg.pull_number",
        "dmg.player_name", "dmg.player_guid", "dmg.player_class",
        "a.player_spec",
        "dmg.total_damage", "dmg.active_time", "dmg.dps",
"f.difficulty", "f.name", "f.start_time", "f.end_time"
    )
)
 

In [0]:
display(damage_with_fights)

In [0]:
# Remove those who switched spec during report
damage_with_fights = damage_with_fights.where(col("player_spec") != "multiple")

# Calculate DPS
damage_with_fights = damage_with_fights.withColumn("fight_duration_sec", (col("end_time") - col("start_time")) / 1000)
gold_player_dps = damage_with_fights.withColumn("damage_per_second", (col("total_damage") / col("fight_duration_sec")).cast("integer") )

In [0]:
display(gold_player_dps)

In [0]:
display(player_dps_gold)